In [1]:
import torch
from unet_vanilla import UNet, RoadSegmentData
import albumentations as A
from albumentations.pytorch import ToTensorV2
import glob, os
from torch.utils.data import Dataset, DataLoader
import torchvision

In [2]:
img_transform = A.Compose(
        [
            ToTensorV2(transpose_mask=True)
        ]
    )

image_names = [img.split('/')[-1] for img in glob.glob("./Vanilla Dataset/test/images/*")]
image_path = './Vanilla Dataset/test/images'
mask_path = './Vanilla Dataset/test/images'
if os.name == 'nt':
    image_names = [img.split('\\')[-1] for img in image_names]
test_data = RoadSegmentData(sorted(image_names), image_path, mask_path, img_transform)
test_dataloader = DataLoader(test_data, batch_size=1, shuffle=False)

In [3]:
torch.cuda.empty_cache() 
model = torch.load('./unet_eth_data_only')
save_path = './fin_unet_eth_only'
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [4]:
model.cuda()
model.eval()

UNet(
  (enc_layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1):

In [5]:
with torch.no_grad():
    for batch, (img,(x, _)) in enumerate(zip(image_names, test_dataloader)):
            x = x.float().cuda()
            y_pred = model(x)
            torchvision.utils.save_image(y_pred[0], save_path + f'/{img}')